In [ ]:
from pulp import *

# Dados do problema
projetos = {
    1: {'ano0': 5, 'ano1': 4, 'ano2': 6, 'vpl': 22},
    2: {'ano0': 4, 'ano1': 3, 'ano2': 3, 'vpl': 17},
    3: {'ano0': 3, 'ano1': 2, 'ano2': 1, 'vpl': 16},
    4: {'ano0': 4, 'ano1': 1, 'ano2': 2, 'vpl': 14},
    5: {'ano0': 5, 'ano1': 3, 'ano2': 6, 'vpl': 20}
}

recursos_disponiveis = {'ano0': 12, 'ano1': 10, 'ano2': 9}

# Cria o problema de maximização
prob = LpProblem("Investiilia_Orcamento_Capital", LpMaximize)

# Variáveis de decisão (binárias: 1 se projeto for selecionado, 0 caso contrário)
x = LpVariable.dicts("Projeto", projetos.keys(), cat='Binary')

# Função objetivo: Maximizar o VPL total
prob += lpSum([projetos[j]['vpl'] * x[j] for j in projetos])

# Restrições de recursos por ano
prob += lpSum([projetos[j]['ano0'] * x[j] for j in projetos]) <= recursos_disponiveis['ano0'], "Recurso_Ano0"
prob += lpSum([projetos[j]['ano1'] * x[j] for j in projetos]) <= recursos_disponiveis['ano1'], "Recurso_Ano1"
prob += lpSum([projetos[j]['ano2'] * x[j] for j in projetos]) <= recursos_disponiveis['ano2'], "Recurso_Ano2"

# Resolve o problema
prob.solve()

# Exibe os resultados
print("Status:", LpStatus[prob.status])
print("VPL Total Máximo (R$ milhões):", value(prob.objective))
print("\nProjetos Selecionados:")
for j in projetos:
    if x[j].value() == 1:
        print(f"Projeto {j}")

print("\nRecursos Utilizados:")
print(f"Ano 0: {sum(projetos[j]['ano0'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano0']}")
print(f"Ano 1: {sum(projetos[j]['ano1'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano1']}")
print(f"Ano 2: {sum(projetos[j]['ano2'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano2']}")

# Análise Detalhada do Código de Otimização para Investiilia

Vou explicar cada parte do código Python que resolve o problema de orçamento de capital da Investiilia, utilizando a biblioteca PuLP.

## 1. Estruturação dos Dados

```python
projetos = {
    1: {'ano0': 5, 'ano1': 4, 'ano2': 6, 'vpl': 22},
    2: {'ano0': 4, 'ano1': 3, 'ano2': 3, 'vpl': 17},
    ...
}
recursos_disponiveis = {'ano0': 12, 'ano1': 10, 'ano2': 9}
```

- **Dicionário de projetos**: Cada projeto é armazenado como um dicionário dentro de um dicionário maior, onde:
  - As chaves são números (1 a 5) identificando cada projeto
  - Os valores são outro dicionário com os investimentos anuais e o VPL
- **Recursos disponíveis**: Um dicionário separado armazena os limites orçamentários para cada ano

## 2. Criação do Problema de Programação

```python
prob = LpProblem("Investiilia_Orcamento_Capital", LpMaximize)
```

- `LpProblem` cria um novo problema de programação linear
- Parâmetros:
  - Nome do problema ("Investiilia_Orcamento_Capital")
  - Tipo de otimização (`LpMaximize` pois queremos maximizar o VPL)

## 3. Variáveis de Decisão

```python
x = LpVariable.dicts("Projeto", projetos.keys(), cat='Binary')
```

- `LpVariable.dicts` cria variáveis de decisão para cada projeto
- Parâmetros:
  - Prefixo do nome ("Projeto")
  - Chaves do dicionário de projetos (`projetos.keys()`)
  - Tipo das variáveis (`cat='Binary'` pois são variáveis 0-1)

## 4. Função Objetivo

```python
prob += lpSum([projetos[j]['vpl'] * x[j] for j in projetos])
```

- `lpSum` calcula a soma linear dos termos
- Para cada projeto `j`, multiplicamos seu VPL (`projetos[j]['vpl']`) pela variável de decisão `x[j]`
- O operador `+=` adiciona esta expressão como função objetivo ao problema

## 5. Restrições Orçamentárias

```python
prob += lpSum([projetos[j]['ano0'] * x[j] for j in projetos]) <= recursos_disponiveis['ano0'], "Recurso_Ano0"
```

- Similar à função objetivo, mas para cada ano:
  - Calcula o total investido no ano (soma dos investimentos dos projetos selecionados)
  - Adiciona a restrição de que deve ser ≤ ao recurso disponível
  - O último parâmetro é um nome descritivo para a restrição

## 6. Resolução do Problema

```python
prob.solve()
```

- Chama o solver padrão do PuLP para resolver o problema
- O PuLP automaticamente seleciona um solver disponível (CBC, GLPK, etc.)

## 7. Exibição dos Resultados

```python
print("Status:", LpStatus[prob.status])
print("VPL Total Máximo (R$ milhões):", value(prob.objective))
```

- `LpStatus` mostra o status da solução (Ótima, Inviável, etc.)
- `value(prob.objective)` retorna o valor da função objetivo na solução ótima

```python
for j in projetos:
    if x[j].value() == 1:
        print(f"Projeto {j}")
```

- Itera por todos os projetos
- Verifica quais variáveis de decisão têm valor 1 (projetos selecionados)
- Imprime os projetos selecionados

```python
print(f"Ano 0: {sum(projetos[j]['ano0'] * x[j].value() for j in projetos)}/{recursos_disponiveis['ano0']}")
```

- Para cada ano, calcula o total de recursos utilizados
- Compara com o limite disponível (formato "usado/limite")